In [1]:
import pandas as pd
import numpy as np
import math as math
import scipy.stats as sts
import scipy.optimize as opt
import numpy.linalg as lin
import statsmodels.api as sm

In [2]:
#Opening files, weeks to use are from 173 to 224 both included

refj_mov = pd.read_stata("Dataset/WRFJ.DTA")
refj_upc = pd.read_csv('Dataset/upcrfj.csv', delimiter=',', header=0) 
upc_f = pd.ExcelFile("Dataset/UPC_LIST_RJ.xlsx")
upc_filtered = upc_f.parse("upcrfj")
demo = pd.read_stata("Dataset/demo.dta")
count = pd.read_stata("Dataset/ccount.dta")

In [3]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('float_format', '{:f}'.format)

In [4]:
# slicing data

rj = refj_mov.loc[lambda rj: refj_mov.week<225, :]
rjf = rj.loc[lambda rj2: rj.week>172, :]
rjf_ok = rjf.loc[lambda rj3: rjf.ok==1, :]

count2 = count.loc[:, lambda count: ['store', 'week', 'custcoun', 'date']]
count3 = count2.loc[lambda count2: count2.week>172, :]
countf = count3.loc[lambda count3: count3.week<225, :]

demof = demo.loc[:, lambda demo: ['store', 'zone', 'income', 'AGE60', 'ethnic', 'hvalmean', 'shopindx', 'cubdist', 'omnidist', 'sstrdist', 'dtdist']]

In [5]:
zones = demof.loc[lambda zones: demof.zone>=1,:]
zones.dropna()
zones.sort_values(by=['zone', 'store'])

rjf_ok.loc[:,'zone']=rjf_ok['store'].map(zones.set_index('store')['zone'])

C:\Users\Daniela\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Daniela\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [6]:
rjf_ok.loc[:,'income']=rjf_ok['store'].map(zones.set_index('store')['income'])
rjf_ok.loc[:,'AGE60']=rjf_ok['store'].map(zones.set_index('store')['AGE60'])
rjf_ok.loc[:,'ethnic']=rjf_ok['store'].map(zones.set_index('store')['ethnic'])
rjf_ok.loc[:,'hvalmean']=rjf_ok['store'].map(zones.set_index('store')['hvalmean'])
rjf_ok.loc[:,'shopindx']=rjf_ok['store'].map(zones.set_index('store')['shopindx'])
rjf_ok.loc[:,'cubdist']=rjf_ok['store'].map(zones.set_index('store')['cubdist'])
rjf_ok.loc[:,'omnidist']=rjf_ok['store'].map(zones.set_index('store')['omnidist'])
rjf_ok.loc[:,'sstrdist']=rjf_ok['store'].map(zones.set_index('store')['sstrdist'])
rjf_ok.loc[:,'dtdist']=rjf_ok['store'].map(zones.set_index('store')['dtdist'])

In [7]:
rjf_ok.loc[:,'desc']=rjf_ok['upc'].map(refj_upc.set_index('UPC')['Desc']).fillna(0)
rjf_ok.loc[:,'OZ']=rjf_ok['upc'].map(refj_upc.set_index('UPC')['OZ'])

In [8]:
countf.loc[:,'store-week'] = countf['store'].apply(lambda x: x*1000)+countf['week']
countf_perweek = countf.groupby(['store','week','store-week'], as_index=False)['custcoun'].sum()

In [9]:
rjf_ok.loc[:,'store-week'] = rjf_ok['store'].apply(lambda x: x*1000)+rjf_ok['week']
rjf_ok.loc[:,'totcount']=rjf_ok['store-week'].map(countf_perweek.set_index('store-week')['custcoun']).fillna(0)

In [10]:
rjf_ok=rjf_ok.dropna()

In [11]:
promo_dummies_rj = pd.get_dummies(rjf_ok['sale'])
promo_drj= promo_dummies_rj.B+promo_dummies_rj.G+promo_dummies_rj.S
promo_drj=pd.DataFrame(promo_drj)
promo_drj.columns=['promo']
rjf_ok.loc[:,'promo']=promo_drj

In [12]:
rjf_ok.loc[:,'family']=round(rjf_ok.upc/1e6,0)

In [13]:
rjfbrand_d = pd.get_dummies(rjf_ok['family'])

In [14]:
rjfbrand_d.columns = ['o1', 'florida', 'o2', 'MinMade', 'o3', 'Domm', 'o4', 'o5', 'o6', 'Tropic', 'o7', 'o8', 'o9','o10','o11','o12','o13','o14','o15','o16']
rjfbrand_d2 = rjfbrand_d[['florida','MinMade','Domm','Tropic']]

In [15]:
rjf_ok=pd.concat([rjf_ok,rjfbrand_d2], axis=1)

In [16]:
rjf_ok['share'] = rjf_ok.move/rjf_ok.totcount

In [17]:
total = rjf_ok.move.sum()
#all_d=lndf_ok.all_det[200]

In [18]:
florida=rjf_ok.move*rjf_ok.florida
MinMade=rjf_ok.move*rjf_ok.MinMade
Domm=rjf_ok.move*rjf_ok.Domm
Tropic=rjf_ok.move*rjf_ok.Tropic
total=florida.sum()+MinMade.sum()+Domm.sum()+Tropic.sum()

In [19]:
print(florida.sum()/total)
print(MinMade.sum()/total)
print(Domm.sum()/total)
print(Tropic.sum()/total)

0.048737018585539846
0.2867739930171557
0.28701597428627396
0.3774730141110305


In [20]:
size_dummies_rj = pd.get_dummies(rjf_ok['OZ'])
#size_dummies_rj[' 64']
size_64_d_rj = size_dummies_rj[' 64']
rjf_ok=pd.concat([rjf_ok,size_64_d_rj], axis=1)

In [21]:
#rjf_ok=rjf_ok.drop([' 64'], axis=1)
rjf_ok.rename(columns={' 64':'64_OZ'})
""

''

In [22]:
rjf_ok = rjf_ok.loc[lambda rj3: rjf_ok.move>0, :]

In [23]:
florida64 = upc_filtered.iloc[:2,:4]
MinMade64 = upc_filtered.iloc[2:6,:4]
MinMade96 = upc_filtered.iloc[6:8,:4]
HH64 = upc_filtered.iloc[19:21,:4]
HH128 = upc_filtered.iloc[21:,:4]
TropPr64 = upc_filtered.iloc[14:16,:4]
TropPr96 = upc_filtered.iloc[16,:4]
TropSB64 = upc_filtered.iloc[17:19,:4]

In [24]:
florida64_d1=rjf_ok['upc'].isin(florida64['upc'])
florida64_d2=pd.get_dummies(florida64_d1)
rjf_ok['florida'] = florida64_d2.iloc[:,1]
print("florida64: ",(florida64_d2.iloc[:,1]*rjf_ok['move']).sum())

florida64:  129325


In [25]:
MinMade64_d1=rjf_ok['upc'].isin(MinMade64['upc'])
MinMade64_d2=pd.get_dummies(MinMade64_d1)
MinMade96_d1=rjf_ok['upc'].isin(MinMade96['upc'])
MinMade96_d2=pd.get_dummies(MinMade96_d1)
rjf_ok['MinMade'] = MinMade64_d2.iloc[:,1]+MinMade96_d2.iloc[:,1]
print("MinMade64: ",(MinMade64_d2.iloc[:,1]*rjf_ok['move']).sum())
print("MinMade96: ",(MinMade96_d2.iloc[:,1]*rjf_ok['move']).sum())

MinMade64:  1026944
MinMade96:  71721


In [26]:
HH64_d1=rjf_ok['upc'].isin(HH64['upc'])
HH64_d2=pd.get_dummies(HH64_d1)
HH128_d1=rjf_ok['upc'].isin(HH128['upc']) # I used 128 OZ because it is the only different size from 64. no Dom in Dataset
HH128_d2=pd.get_dummies(HH128_d1)
rjf_ok['HH'] = HH64_d2.iloc[:,1]+HH128_d2.iloc[:,1]
print("HH64: ",(HH64_d2.iloc[:,1]*rjf_ok['move']).sum())
print("HH128(96): ",(HH128_d2.iloc[:,1]*rjf_ok['move']).sum())

HH64:  1352914
HH128(96):  285351


In [27]:
TropPr64_d1=rjf_ok['upc'].isin(TropPr64['upc'])
TropPr64_d2=pd.get_dummies(TropPr64_d1)
TropPr96_d1=rjf_ok['upc'].isin([TropPr96['upc']])
TropPr96_d2=pd.get_dummies(TropPr96_d1)
rjf_ok['Tropic'] = TropPr64_d2.iloc[:,1]+TropPr96_d2.iloc[:,1]
print("TropPr64: ",(TropPr64_d2.iloc[:,1]*rjf_ok['move']).sum())
print("TropPr96: ",(TropPr96_d2.iloc[:,1]*rjf_ok['move']).sum())

TropPr64:  639779
TropPr96:  176850


In [28]:
TropSB64_d1=rjf_ok['upc'].isin(TropSB64['upc'])
TropSB64_d2=pd.get_dummies(TropSB64_d1)
rjf_ok['TropicSB'] = TropSB64_d2.iloc[:,1]
print("TropSB64: ",(TropSB64_d2.iloc[:,1]*rjf_ok['move']).sum())


TropSB64:  741646


In [29]:
rjf_ok['64 OZ']=florida64_d2.iloc[:,1]+MinMade64_d2.iloc[:,1]+HH64_d2.iloc[:,1]+TropPr64_d2.iloc[:,1]+TropSB64_d2.iloc[:,1]
rjf_ok['96 OZ']=MinMade96_d2.iloc[:,1]+HH128_d2.iloc[:,1]+TropPr96_d2.iloc[:,1]
rjf_ok['nobrand']=1-rjf_ok['64 OZ']-rjf_ok['96 OZ'] #nobrand=1 other brands


In [30]:
#rjf_ok.iloc[:5,15:]
#rjf_ok=rjf_ok.drop(columns=['Domm', ' 64'], axis=1)

In [31]:
rjf_ok = rjf_ok.loc[lambda rjnb: rjf_ok.nobrand==0, :]

In [32]:
florida=rjf_ok.move*rjf_ok.florida
MinMade=rjf_ok.move*rjf_ok.MinMade
HH=rjf_ok.move*rjf_ok.HH
Tropic=rjf_ok.move*rjf_ok.Tropic
TropicSB=rjf_ok.move*rjf_ok.TropicSB
total=florida.sum()+MinMade.sum()+HH.sum()+Tropic.sum()+TropicSB.sum()

In [33]:
print(florida.sum()/total)
print(MinMade.sum()/total)
print(HH.sum()/total)
print(Tropic.sum()/total)
print(TropicSB.sum()/total)

0.029229093259623057
0.24831225011470145
0.3702687065066798
0.1845685304427815
0.1676214196762142


In [116]:
#Price, and cost, Descriptive Statistics

AvgP_f64 = np.sum(rjf_ok['florida']*rjf_ok['64 OZ']*rjf_ok['price']*rjf_ok['move'])/np.sum(rjf_ok['florida']*rjf_ok['64 OZ']*rjf_ok['move'])
AvgC_f64 = np.sum(rjf_ok['florida']*rjf_ok['64 OZ']*(rjf_ok['price']*(1-rjf_ok['profit']/100))*rjf_ok['move'])/np.sum(rjf_ok['florida']*rjf_ok['64 OZ']*rjf_ok['move'])

AvgP_m64 = np.sum(rjf_ok['MinMade']*rjf_ok['64 OZ']*rjf_ok['price']*rjf_ok['move'])/np.sum(rjf_ok['MinMade']*rjf_ok['64 OZ']*rjf_ok['move'])
AvgC_m64 = np.sum(rjf_ok['MinMade']*rjf_ok['64 OZ']*(rjf_ok['price']*(1-rjf_ok['profit']/100))*rjf_ok['move'])/np.sum(rjf_ok['MinMade']*rjf_ok['64 OZ']*rjf_ok['move'])

AvgP_m96 = np.sum(rjf_ok['MinMade']*rjf_ok['96 OZ']*rjf_ok['price']*rjf_ok['move'])/np.sum(rjf_ok['MinMade']*rjf_ok['96 OZ']*rjf_ok['move'])
AvgC_m96 = np.sum(rjf_ok['MinMade']*rjf_ok['96 OZ']*(rjf_ok['price']*(1-rjf_ok['profit']/100))*rjf_ok['move'])/np.sum(rjf_ok['MinMade']*rjf_ok['96 OZ']*rjf_ok['move'])

AvgP_h64 = np.sum(rjf_ok['HH']*rjf_ok['64 OZ']*rjf_ok['price']*rjf_ok['move'])/np.sum(rjf_ok['HH']*rjf_ok['64 OZ']*rjf_ok['move'])
AvgC_h64 = np.sum(rjf_ok['HH']*rjf_ok['64 OZ']*(rjf_ok['price']*(1-rjf_ok['profit']/100))*rjf_ok['move'])/np.sum(rjf_ok['HH']*rjf_ok['64 OZ']*rjf_ok['move'])

AvgP_h96 = np.sum(rjf_ok['HH']*rjf_ok['96 OZ']*rjf_ok['price']*rjf_ok['move'])/np.sum(rjf_ok['HH']*rjf_ok['96 OZ']*rjf_ok['move'])
AvgC_h96 = np.sum(rjf_ok['HH']*rjf_ok['96 OZ']*(rjf_ok['price']*(1-rjf_ok['profit']/100))*rjf_ok['move'])/np.sum(rjf_ok['HH']*rjf_ok['96 OZ']*rjf_ok['move'])

AvgP_t64 = np.sum(rjf_ok['Tropic']*rjf_ok['64 OZ']*rjf_ok['price']*rjf_ok['move'])/np.sum(rjf_ok['Tropic']*rjf_ok['64 OZ']*rjf_ok['move'])
AvgC_t64 = np.sum(rjf_ok['Tropic']*rjf_ok['64 OZ']*(rjf_ok['price']*(1-rjf_ok['profit']/100))*rjf_ok['move'])/np.sum(rjf_ok['Tropic']*rjf_ok['64 OZ']*rjf_ok['move'])

AvgP_t96 = np.sum(rjf_ok['Tropic']*rjf_ok['96 OZ']*rjf_ok['price']*rjf_ok['move'])/np.sum(rjf_ok['Tropic']*rjf_ok['96 OZ']*rjf_ok['move'])
AvgC_t96 = np.sum(rjf_ok['Tropic']*rjf_ok['96 OZ']*(rjf_ok['price']*(1-rjf_ok['profit']/100))*rjf_ok['move'])/np.sum(rjf_ok['Tropic']*rjf_ok['96 OZ']*rjf_ok['move'])

AvgP_tsb64 = np.sum(rjf_ok['TropicSB']*rjf_ok['64 OZ']*rjf_ok['price']*rjf_ok['move'])/np.sum(rjf_ok['TropicSB']*rjf_ok['64 OZ']*rjf_ok['move'])
AvgC_tsb64 = np.sum(rjf_ok['TropicSB']*rjf_ok['64 OZ']*(rjf_ok['price']*(1-rjf_ok['profit']/100))*rjf_ok['move'])/np.sum(rjf_ok['TropicSB']*rjf_ok['64 OZ']*rjf_ok['move'])

print("Avg. Prices: F64: ", AvgP_f64, "M64",AvgP_m64 , "M96", AvgP_m96, "H64", AvgP_h64, "H96", AvgP_h96, "T64", AvgP_t64, "T96", AvgP_t96, "TSB64", AvgP_tsb64)
print("Avg. Costs: F64: ", AvgC_f64, "M64",AvgC_m64 , "M96", AvgC_m96, "H64", AvgC_h64, "H96", AvgC_h96, "T64", AvgC_t64, "T96", AvgC_t96, "TSB64", AvgC_tsb64)

Avg. Prices: F64:  2.3090356852890004 M64 1.798598842780132 M96 3.4540322917973816 H64 1.3399150130754798 H96 3.1722692753836506 T64 2.3887041775363054 T96 4.165218659881255 TSB64 1.6379981824212628
Avg. Costs: F64:  1.5746899974869515 M64 1.5280160532998879 M96 2.537460605526973 H64 0.8633268559065839 H96 2.029978377016445 T64 1.7292947001652132 T96 3.068523378360192 TSB64 1.3871188670093277


In [34]:
def InstVar(price, instruments):
    first_stage = sm.OLS(price,instruments).fit()
    coeffs = first_stage.params
    price_est = instruments@coeffs
    price_estf = np.reshape(price_est,(price_est.size,1))
    return price_estf

# Estimation of demand

In [35]:
"""
Variables to be estimated:
    
alpha => alpha_j (with j for each brand), mean intrinsic utility across consumers for brand and size x

sigma => sigma_income, sigma_age, sigma_ethnic, sigma_hval, sigma_shopindx, sigma_edlpd: deviations 
from the mean due to demographic variables (common values across stores)

beta => beta_promo, beta_size: mean coefficients for observed attributes
    
lambda_p&s => lambda_promo, lambda_size: deviation coefficients for for observed attributes
    
theta and gamma => theta_price, gamma_income, gamma_age, gamma_ethnic, gamma_hval, gamma_shopindx, gamma_edlpd: coefficients for price
    
lambda_price => lambda_price: coefficient for the deviation of price

l_matrix => l1_j, l2_j = coefficients of the latent attributes for each brand 

draws are relative to the standard deviations coefficients
"""
print( )

In [36]:
"""main equations:
    
delta(alpha, beta, sigma, gamma, promo and size, demog_vars, brand_dummies, price, xi): this function calculates the mean utility 
for each brand and store. 

delta_jts = brand_dummies_ts@alpha_j+ promo&size@beta + demog_vars_s@sigma \
            - (theta+demog_vars_s@gamma)*price_jts + xi // vec. of obsx1

mu(promo&size, price, lambda_p&s, lambda_price, l_matrix, simulation draws): this function calculates the deviations from 
the mean, using independent draws."""
print( )

In [37]:
def delta_fun (alpha, beta, sigma, gamma, theta, xi, data_promo_size, data_demog, brand_dummies, price, store_week):
    a = np.reshape(alpha,(alpha.size,1))
    b = np.reshape(beta,(beta.size,1))
    s = np.reshape(sigma,(sigma.size,1))
    g = np.reshape(gamma,(gamma.size,1))
    ps = data_promo_size
    dd = data_demog
    bd = brand_dummies
    p = price
    mkt = np.reshape(store_week,(store_week.size,1))
    
    delta_mat = np.zeros((mkt.shape[0], 2))
    delta_mat[:,0] = mkt[:,0]
    trd = dd@s + bd@a +ps@b + (theta+dd@g)*p    #+ xi[:,0] # obs x 2 matrix, 2nd column vector of deltas
    delta_mat[:,1] = trd[:,0]
    return delta_mat

def mu_fun(data_promo_size, price, brand_dummies, L_matrix, lambda_x,lambda_price, sim_beta, sim_theta, sim_w):
    
    ps = np.reshape(data_promo_size,(data_promo_size.shape[0],data_promo_size.shape[1]))
    p = np.reshape(price,(price.size,1))
    bd = brand_dummies

    lambda_x = np.reshape(lambda_x,(lambda_x.size,1))
    L_matrix = L_matrix
    
    sim_beta = sim_beta
    sim_theta = np.reshape(sim_theta,(sim_theta.size,1))
    sim_w = sim_w
    
    sim_mat = np.hstack((sim_beta*lambda_x.T, lambda_price*sim_theta,sim_w@L_matrix.T)) # S x (J+3) matrix
    param_mat = np.hstack((ps,price,brand_dummies))  # obs x (J+3) matrix
    
    mu_mat = param_mat@sim_mat.T #obsxS
    return mu_mat

In [38]:
#contraction mapping

def est_ms_cm(delta1, theta2, data, simP):
    sim_beta, sim_theta, sim_w = simP
    data_promo_size, data_demog, brand_dummies, price, store_week, share = data
    L_matrix, lambda_x,lambda_price = theta2

    d=delta1
    mu = mu_fun(data_promo_size, price, brand_dummies,L_matrix, lambda_x,lambda_price,sim_beta, sim_theta, sim_w) # obs x S

    delta = d[:,1]
    delta = np.reshape(np.asarray(delta),(price.size,1))
    share = np.reshape(np.asarray(share),(share.size,1))
    delta = np.exp(delta)
    mu = np.exp(mu)
    market = np.reshape(store_week,(store_week.size,1))

    share_mat = delta*mu
    share_mat = np.hstack((market,share_mat))
    share_mat2 = pd.DataFrame((share_mat))
    
    tot_share_mat = share_mat2.groupby(share_mat2.iloc[:,0]).sum()
    tot_share_mat = tot_share_mat + 1
    
    share_mat2=share_mat2.rename(columns={0:'market'})
    share_mat2.sort_values(by=['market'])
    
    r2=share_mat2.iloc[0,0]
    a=(tot_share_mat.loc[r2,1:]).values.reshape(1,30)
    i=0
    for i in range(share_mat2.shape[0]):
        r=share_mat2.iloc[i,0]
        if r==r2:
            share_mat2.iloc[i,1:]=(share_mat2.iloc[i,1:]).values.reshape(1,30)/a #tot_share_mat.loc[r,:]
        else:
            r2=share_mat2.iloc[i,0]
            a=(tot_share_mat.loc[r2,1:]).values.reshape(1,30)
            share_mat2.iloc[i,1:]=(share_mat2.iloc[i,1:]).values.reshape(1,30)/a
    
    ns = mu.shape[1]
    est_shares_vec = np.zeros((d.shape[0],1))
    est_shares_vec[:,0] = (share_mat2.iloc[:,1:].sum(axis=1)).apply(lambda x: x*(1/ns))
    new_delta_3 = d[:,1] + np.log(share[:,0]) - np.log(est_shares_vec[:,0])
    d[:,1] = new_delta_3
    new_delta = d
    return new_delta, est_shares_vec

In [39]:
#This function generates the new deltas for the iteration.

def delta_eq(d_in,theta2, data, simP):
    data_promo_size, data_demog, brand_dummies, price, store_week, share = data
 
    delta_new, estim_shares_new = est_ms_cm(d_in, theta2, data, simP) # estimated shares are not used in this cell.

    diff_sh = abs(np.log(share) - np.log(estim_shares_new))
    epsilon = 1e-8
    i=0
    while diff_sh.all() >= epsilon:
        delta_ant=delta_new
        delta_new, est_shares_new = est_ms_cm(delta_ant, theta2, data, simP)
        diff_sh = abs(np.log(share) - np.log(est_shares_new))
        print(diff_sh.max())
        i+=1

    delta_eq = delta_new
    
    return delta_eq

In [40]:
#this function calculates the error vector (xi_jt)

def errorvec(delta_inic, data, theta1, theta2, xi, simP):
    
    alpha, beta, sigma, gamma, theta = theta1
    data_promo_size, data_demog, brand_dummies, price, store_week, share = data
    delta_equiv = delta_eq(delta_inic, theta2,data, simP)

    delta_mod = delta_fun(alpha, beta, sigma, gamma, theta, xi, data_promo_size, data_demog, brand_dummies, price, store_week)

    error_vec = delta_equiv[:,1] - delta_mod[:,1]
    
    return error_vec

In [41]:
def logs(market, share_obs):
    storeweek = np.array(market)
    share = np.array(share_obs)
    ts=np.zeros((share.shape[0],1))
    share2=np.hstack((storeweek,share,ts))
    share2=pd.DataFrame((share2))
    tot_shares_real=share2.groupby(share2.iloc[:,0]).sum()
    i=0
    for i in range(share.shape[0]):
        r=share2.iloc[i,0]
        tots=tot_shares_real.loc[r,:]
        share2.iloc[i,2]=1-tots[1]
        i+=1
    logs=np.asarray(np.log(share2.iloc[:,1])-np.log(share2.iloc[:,2]))
    logs=np.reshape(logs, (logs.size,1))
    return logs

In [42]:
def paramsnohet(datax, theta1):
    alpha, beta, sigma, gamma, theta = theta1
    data_promo_size, data_demog, brand_dummies, price, store_week, share = datax
    
    a = np.reshape(alpha,(alpha.size,1))
    b = np.reshape(beta,(beta.size,1))
    s = np.reshape(sigma,(sigma.size,1))
    g = np.reshape(gamma,(gamma.size,1))
    ps = data_promo_size
    dd = data_demog
    bd = brand_dummies
    p = price
    mkt = np.reshape(store_week,(store_week.size,1))
    
    Y = logs(mkt, share)
    X=np.hstack((ps, dd, p, bd, dd*p))
    X=pd.DataFrame(X)
    X.columns=['promo', '64 OZ', 'income', 'AGE60', 'ethnic', 'hvalmean','shopindx','sstrdist', 'price_e',\
              'florida','MinMade','HH','Tropic','TropicSB', 'inc*p', 'AGE*p', 'eth*p', 'hval*p', 'shop*p', 'EDLP*p']
    
    trd = dd@s + bd@a +ps@b + (theta+dd@g)*p
    
    par_nohet = sm.OLS(Y,X).fit()
    return trd.shape, par_nohet.summary()

In [54]:
#GMM code
def GMM_criterion(params, *args):
    theta1, del_in, data, xi, simP, instruments = args
    a1,a2,a3,a4,a5,b1,b2,s1,s2,s3,s4,s5,s6,g1,g2,g3,g4,g5,g6, theta,\
    L11,L21,L22,L31,L32,L41,L42,L51,L52,lx1,lx2,lambda_price = params
    data_promo_size, data_demog, brand_dummies, price, store_week, share = data
    
    alpha=np.array([a1,a2,a3,a4,a5])
    beta=np.array([b1,b2])
    gamma = np.array([g1,g2,g3,g4,g5,g6])
    sigma = np.array([s1,s2,s3,s4,s5,s6])
    L_matrix=np.array([[L11,0],[L21,L22],[L31,L32],[L41,L42],[L51,L52]])
    lambda_x = np.array([lx1,lx2])
    
    theta1=(alpha, beta, sigma, gamma, theta)
    theta2=(L_matrix, lambda_x, lambda_price)
    Z=instruments
    mat_inv = lin.inv(Z.T@Z)
    error_vec = errorvec(del_in, data, theta1, theta2, xi, simP)
    GMM = error_vec.T@Z@mat_inv@Z.T@error_vec
    print("GMM: ",GMM)
    return GMM

### List of parameters, variables and draws:

#### Case 2: Refrigerated Orange Juice

- Brands: florida, MinuteMade, HH, Tropicana Premium, Tropicana SB (Dominick's own brand information not available)
- Sizes: 64 OZ and 96 OZ

##### Variables:

- price: rows: # obs, columns: weekly price per product per store
- data_demog: rows: # obs, columns: income, age, ethnic, hval, shopindx, edlpd
- brand_dummies: rows: # obs, columns: 5 columns of dummies for florida, MinuteMade, HH, Tropicana Premium, Tropicana SB respectively
- data_promo_size: rows: # obs, columns: 2 columns of dummies for promo (1 if the price includes promo) and size (64 OZ = 1)
- storeweek: rows: # obs, columns: ID for each market (built from original columns (store x 1,000) + week)
- share: rows: # obs, columns: absolute share (also considering outside opt) for each product in each market.

##### Parameters:

- alpha: rows: 5, columns: 1 => alpha_florida, alpha_MinMade, alpha_HH, alpha_TropicPr, alpha_TropicSB
- beta: rows: 2, columns: 1 => beta_promo, beta_size
- sigma: rows: 6, columns: 1 => sigma_income, sigma_age, sigma_ethnic, sigma_hval, sigma_shopindx, sigma_edlpd
- gamma: rows: 6, columns: 1 => gamma_income, gamma_age, gamma_ethnic, gamma_hval, gamma_shopindx, gamma_edlpd
- theta: 1 parameter, scalar
- L_matrix: rows: 5, columns: 2 => [[L1_florida, L2_florida],[L1_MinMade, L2_MinMade], [L1_HH, L2_HH], [L1_TropPr, L2_TropPr], [L1_TropSB, L2_TropSB]].
- lambda_x: rows: 2, columns: 1 => lambda_promo, lambda_size
- lambda_price: 1 parameter, scalar

##### Draws:

Normal Standard draws.

- sim_beta: rows: #Draws, columns: 2 => sim_beta_promo, sim_beta_size. Draws for the standard deviation of promo and size 
- sim_theta: rows: #Draws, columns: 1 => sim_theta. Draws for the theta parameter relative to price
- sim_w: rows: #Draws, columns: 2 => sim_w1, sim_w2. Draws for the w vector used to calculate the Sigma matrix

In [44]:
# Creating uniform and standard normal draws

sim_draws_beta = 30
sim_draws_theta = 30
sim_draws_w = 30
np.random.seed(1979)

uniform_beta = sts.uniform.rvs(0, 1, size=(sim_draws_beta,2))
normal_beta_draws = sts.norm.ppf(uniform_beta, loc=0, scale=1)

uniform_theta = sts.uniform.rvs(0, 1, size=(sim_draws_theta, 1))
normal_theta_draws = sts.norm.ppf(uniform_theta, loc=0, scale=1)

uniform_w = sts.uniform.rvs(0, 1, size=(sim_draws_w, 2))
normal_w_draws = sts.norm.ppf(uniform_w, loc=0, scale=1)

print(normal_beta_draws.shape, normal_theta_draws.shape, normal_w_draws.shape)

(30, 2) (30, 1) (30, 2)


In [45]:
def delta_ini(market, share_obs):
    storeweek = np.array(market)
    share = np.array(share_obs)
    ts=np.zeros((share.shape[0],1))
    share2=np.hstack((storeweek,share,ts))
    share2=pd.DataFrame((share2))
    tot_shares_real=share2.groupby(share2.iloc[:,0]).sum()
    i=0
    for i in range(share.shape[0]):
        r=share2.iloc[i,0]
        tots=tot_shares_real.loc[r,:]
        share2.iloc[i,2]=1-tots[1]
        i+=1
    logs=np.asarray(np.log(share2.iloc[:,1])-np.log(share2.iloc[:,2]))
    logs=np.reshape(logs, (logs.size,1))
    delta_in=np.hstack((storeweek,logs))
    return delta_in

In [46]:
# preparing the data, grouping it by store-week

demog = np.array(rjf_ok[['income', 'AGE60', 'ethnic', 'hvalmean','shopindx','sstrdist']])
brands = np.array(rjf_ok[['florida','MinMade','HH','Tropic','TropicSB']])
price = np.array(np.log(rjf_ok[['price']]))
wholeprice = np.log(np.array(rjf_ok[['price']])*(1-np.array(rjf_ok[['profit']])/100))
promo=np.reshape(rjf_ok[['promo']],(rjf_ok[['promo']].shape[0],1))
instr=np.hstack((promo,demog, wholeprice, brands))
price_est=InstVar(price,instr)

datapack_GMM = np.hstack((price_est, wholeprice, rjf_ok[['promo', '64 OZ', '96 OZ', 'move','totcount', 'income', 'AGE60',\
                                                          'ethnic', 'hvalmean','shopindx','sstrdist', 'florida','MinMade','HH'\
                                                          ,'Tropic','TropicSB', 'price', 'store-week', 'share', 'store', 'zone', 'week']]))

datapack_GMM=pd.DataFrame(datapack_GMM)
datapack_GMM.columns = ['Price_est', 'wholeprice', 'promo', '64 OZ', '96 OZ', 'move','totcount', 'income', 'AGE60',\
                        'ethnic', 'hvalmean','shopindx','sstrdist', 'florida','MinMade','HH','Tropic',\
                        'TropicSB', 'price', 'store-week', 'share', 'store', 'zone', 'week']
datapack_GMM['64_OZ(2)']=datapack_GMM[['64 OZ']]
datapack_GMM['Price_est']=datapack_GMM['Price_est']*datapack_GMM['move']
datapack_GMM['wholeprice']=datapack_GMM['wholeprice']*datapack_GMM['move']
datapack_GMM = datapack_GMM.groupby(['store-week', 'florida', 'MinMade', 'HH', 'Tropic', 'TropicSB', '64 OZ'], as_index=False).sum()

datapack_GMM['Price_est'] =datapack_GMM['Price_est']/(datapack_GMM['move']) 
datapack_GMM['wholeprice'] =datapack_GMM['wholeprice']/(datapack_GMM['move']) 
datapack_GMM['totcount'] =datapack_GMM['totcount']/(datapack_GMM['96 OZ']+datapack_GMM['64_OZ(2)']) 
datapack_GMM['income'] =datapack_GMM['income']/(datapack_GMM['96 OZ']+datapack_GMM['64_OZ(2)']) 
datapack_GMM['AGE60'] =datapack_GMM['AGE60']/(datapack_GMM['96 OZ']+datapack_GMM['64_OZ(2)']) 
datapack_GMM['ethnic'] =datapack_GMM['ethnic']/(datapack_GMM['96 OZ']+datapack_GMM['64_OZ(2)']) 
datapack_GMM['hvalmean'] =datapack_GMM['hvalmean']/(datapack_GMM['96 OZ']+datapack_GMM['64_OZ(2)']) 
datapack_GMM['shopindx'] =datapack_GMM['shopindx']/(datapack_GMM['96 OZ']+datapack_GMM['64_OZ(2)']) 
datapack_GMM['sstrdist'] =datapack_GMM['sstrdist']/(datapack_GMM['96 OZ']+datapack_GMM['64_OZ(2)']) 
datapack_GMM['price'] =datapack_GMM['price']/(datapack_GMM['96 OZ']+datapack_GMM['64_OZ(2)']) 
datapack_GMM['store'] =datapack_GMM['store']/(datapack_GMM['96 OZ']+datapack_GMM['64_OZ(2)']) 
datapack_GMM['zone'] =datapack_GMM['zone']/(datapack_GMM['96 OZ']+datapack_GMM['64_OZ(2)']) 
datapack_GMM['week'] =datapack_GMM['week']/(datapack_GMM['96 OZ']+datapack_GMM['64_OZ(2)']) 

i=0
for i in range(datapack_GMM.shape[0]):
    if datapack_GMM.loc[i,'promo']>0:
        datapack_GMM.loc[i,'promo'] = 1
    else:
        pass
    i+=1


In [47]:
promosize = np.array(datapack_GMM[['promo', '64 OZ']])
demog = np.array(datapack_GMM[['income', 'AGE60', 'ethnic', 'hvalmean','shopindx','sstrdist']])
brands = np.array(datapack_GMM[['florida','MinMade','HH','Tropic','TropicSB']])
price = np.array(datapack_GMM[['price']])#.astype(np.float16)
storeweek = np.array(datapack_GMM[['store-week']])
share = np.array(datapack_GMM[['share']])
price_est=np.array(datapack_GMM[['Price_est']])
promo = np.array(datapack_GMM[['promo']])
wholeprice = np.array(datapack_GMM[['wholeprice']])

delta_in=delta_ini(storeweek,share)

In [48]:
data = (promosize, demog, brands, price_est, storeweek, share)

alpha=np.array([0,0,0,0,0])
beta=np.array([0,0])
gamma = np.array([0,0,0,0,0,0])
sigma = np.array([0,0,0,0,0,0])
theta = 0

theta1=(alpha, beta, sigma, gamma, theta)

results_nohet=paramsnohet(data, theta1)

In [49]:
results_nohet

((29893, 1), <class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                      y   R-squared:                       0.524
 Model:                            OLS   Adj. R-squared:                  0.524
 Method:                 Least Squares   F-statistic:                     1731.
 Date:                Fri, 22 Mar 2019   Prob (F-statistic):               0.00
 Time:                        09:17:52   Log-Likelihood:                -36183.
 No. Observations:               29893   AIC:                         7.241e+04
 Df Residuals:                   29873   BIC:                         7.257e+04
 Df Model:                          19                                         
 Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
 ---------------------------------------------------------

In [55]:
a1=-8.9642
a2=-7.3520
a3=-6.3738
a4=-7.2199
a5=-7.6181
b1=0.7175
b2=2.0022
s1=-0.0480
s2=2.9029
s3=0.1446
s4=-0.0022
s5=-0.6129
s6=-0.0094
g1=0.5275
g2=-2.7953
g3=0.0173
g4=0.0041
g5= 0.4229
g6=-0.0087

theta = -4.7870

#L12 arbitrarily set to zero
(L11,L21,L22,L31,L32,L41,L42,L51,L52,lx1,lx2,lambda_price)=np.array([0.67759551, 1.03441918, 0.65452123, \
                                                                     0.65452234, 1.02568196, 0.65452186, 0.65452142, \
                                                                     0.65452133, 0.65452136, 0.6545626,  0.65464387, \
                                                                     0.65452742])

alpha=np.array([a1,a2,a3,a4,a5])
beta=np.array([b1,b2])
gamma = np.array([g1,g2,g3,g4,g5,g6])
sigma = np.array([s1,s2,s3,s4,s5,s6])

theta1=(alpha, beta, sigma, gamma, theta)


simP = (normal_beta_draws, normal_theta_draws, normal_w_draws)

params = np.array([a1,a2,a3,a4,a5,b1,b2,s1,s2,s3,s4,s5,s6,g1,g2,g3,g4,g5,g6,theta,\
                   L11,L21,L22,L31,L32,L41,L42,L51,L52,lx1,lx2,lambda_price])

xi = np.zeros((datapack_GMM.shape[0],1))

instruments=np.hstack((promo,demog, wholeprice, brands))

args = (theta1, delta_in, data, xi, simP, instruments)
Bounds = ((None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None),\
          (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None),\
          (None, None), (None, None), (None, None), (None, None), (None, None), (None, None),\
          (1e-10, None), (1e-10, None), (1e-10, None), (1e-10, None), (1e-10, None), (1e-10, None),\
          (1e-10, None), (1e-10, None), (1e-10, None), (1e-10, None), (1e-10, None), (1e-10, None))

In [56]:
results = opt.minimize(GMM_criterion, params, args=(args), method='SLSQP', bounds=Bounds, tol=1e-2) #options={'maxiter': 1}

1.7912618055504481e-09
6.446443379104494e-10
GMM:  14164.046503771799
GMM:  14164.046502127729
GMM:  14164.04658211123
GMM:  14164.046665599062
GMM:  14164.046646394618
GMM:  14164.04661071721
GMM:  14164.046581011598
GMM:  14164.046804848913
GMM:  14164.046937872037
GMM:  14164.052629255988
GMM:  14164.046601885904
GMM:  14164.04659454363
GMM:  14164.129873966433
GMM:  14164.046931300585
GMM:  14164.049555981608
GMM:  14164.051525220919
GMM:  14164.046584049318
GMM:  14164.04657817172
GMM:  14164.115105820223
GMM:  14164.046852867457
GMM:  14164.049037320503
GMM:  14164.046975184952
GMM:  14164.04652717332
1.400978888455029e-09
GMM:  14164.046597268789
1.3825136591094633e-09
GMM:  14164.04655642519
1.2490382061969285e-09
3.5539082787749976e-10
GMM:  14164.046569554608
1.0833773878005104e-09
5.897611288219196e-10
GMM:  14164.046575243574
1.4576135853872074e-09
4.850042589765735e-10
2.642768226479575e-10
GMM:  14164.04654017309
GMM:  14164.046526270275
GMM:  14164.046527618415
9.1720764

In [58]:
a1,a2,a3,a4,a5,b1,b2,s1,s2,s3,s4,s5,s6,g1,g2,g3,g4,g5,g6,theta,L11,L21,L22,L31,L32,L41,L42,L51,L52,lx1,lx2,lambda_price = results.x

results

     fun: 3895.3951455498363
     jac: array([5.36760193e+03, 1.09703755e+04, 9.68158704e+03, 7.28731677e+03,
       5.29380688e+03, 2.03152748e+04, 2.92423055e+04, 4.11184617e+05,
       6.69465784e+03, 6.20192615e+03, 5.59498939e+06, 2.88013029e+04,
       2.04940665e+05, 3.37094078e+05, 5.49766479e+03, 5.10322583e+03,
       4.60391587e+06, 2.35377448e+04, 1.70133907e+05, 3.17463329e+04,
       1.68078113e+03, 6.38480847e+03, 3.64384094e+03, 4.52494128e+03,
       4.90672131e+03, 2.55318103e+03, 1.62017883e+03, 1.71065100e+03,
       1.45505566e+03, 1.32437692e+04, 2.59136600e+04, 5.68537170e+03])
 message: 'Optimization terminated successfully.'
    nfev: 45
     nit: 1
    njev: 1
  status: 0
 success: True
       x: array([-8.96420209e+00, -7.35200426e+00, -6.37380376e+00, -7.21990283e+00,
       -7.61810206e+00,  7.17492107e-01,  2.00218864e+00, -4.81597602e-02,
        2.90289740e+00,  1.44597590e-01, -4.37385751e-03, -6.12911190e-01,
       -9.47962693e-03,  5.27369027e-01, -2

In [59]:
# Calculating the elasticities

alpha_def=np.array([a1, a2, a3, a4, a5])
beta_def=np.array([b1, b2])
sigma_def=np.array([s1, s2, s3, s4, s5, s6])
gamma_def=np.array([g1, g2, g3, g4, g5, g6])
L_matrix_def=np.array([[L11, 0], [L21, L22],  [L31, L32],  \
                   [L41,  L42],  [L51, L52]])
lambda_p_s_def=np.array([lx1, lx2])
theta_def=theta
lambda_price_def=lambda_price

In [60]:
delta_def=delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi, promosize, demog, brands, price_est, storeweek)

In [61]:
mu_def=mu_fun(promosize, price_est, brands, L_matrix_def, lambda_p_s_def,lambda_price_def, normal_beta_draws, normal_theta_draws, normal_w_draws)

In [62]:
#calculating the equilibrium market shares

th2=(L_matrix_def, lambda_p_s_def,lambda_price_def)
data=(promosize, demog, brands, price_est, storeweek,share)
simpack=(normal_beta_draws, normal_theta_draws, normal_w_draws)
d_test, est_sh_test =est_ms_cm(delta_def, th2, data, simpack)

In [63]:
# Price differentials [I NEED TO CREATE PRICES CHANGES FOR 64 AND 96 OZ SEPARATELY]
h=1e-5
P_florida64_diff=np.array(datapack_GMM['Price_est']*(1+h*datapack_GMM['florida']*datapack_GMM['64 OZ']), ndmin=2).T
P_MinMade64_diff=np.array(datapack_GMM['Price_est']*(1+h*datapack_GMM['MinMade']*datapack_GMM['64 OZ']), ndmin=2).T
P_HH64_diff=np.array(datapack_GMM['Price_est']*(1+h*datapack_GMM['HH']*datapack_GMM['64 OZ']), ndmin=2).T
P_Tropic64_diff=np.array(datapack_GMM['Price_est']*(1+h*datapack_GMM['Tropic']*datapack_GMM['64 OZ']), ndmin=2).T
P_TropicSB64_diff=np.array(datapack_GMM['Price_est']*(1+h*datapack_GMM['TropicSB']*datapack_GMM['64 OZ']), ndmin=2).T

#P_florida96_diff=np.array(datapack_GMM['Price_est']*(1+h*datapack_GMM['florida']*datapack_GMM['96 OZ']), ndmin=2).T
P_MinMade96_diff=np.array(datapack_GMM['Price_est']*(1+h*datapack_GMM['MinMade']*datapack_GMM['96 OZ']), ndmin=2).T
P_HH96_diff=np.array(datapack_GMM['Price_est']*(1+h*datapack_GMM['HH']*datapack_GMM['96 OZ']), ndmin=2).T
P_Tropic96_diff=np.array(datapack_GMM['Price_est']*(1+h*datapack_GMM['Tropic']*datapack_GMM['96 OZ']), ndmin=2).T
#P_TropicSB96_diff=np.array(datapack_GMM['Price_est']*(1+h*datapack_GMM['TropicSB']*datapack_GMM['96 OZ']), ndmin=2).T

In [64]:
#delta function with price differentials (TropicSB 96 OZ is not included in the paper)

d_florida64=delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi, promosize, demog, brands, P_florida64_diff, storeweek)
d_MinMade64=delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi, promosize, demog, brands, P_MinMade64_diff, storeweek)
d_HH64=delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi, promosize, demog, brands, P_HH64_diff, storeweek)
d_Tropic64=delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi, promosize, demog, brands, P_Tropic64_diff, storeweek)
d_TropicSB64=delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi, promosize, demog, brands, P_TropicSB64_diff, storeweek)

#d_florida96=delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi, promosize, demog, brands, P_florida96_diff, storeweek)
d_MinMade96=delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi, promosize, demog, brands, P_MinMade96_diff, storeweek)
d_HH96=delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi, promosize, demog, brands, P_HH96_diff, storeweek)
d_Tropic96=delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi, promosize, demog, brands, P_Tropic96_diff, storeweek)


In [65]:
P_florida64_diff.shape

(29893, 1)

In [66]:
# calculating the estimated market shares upon a change in price

data_florida64=(promosize, demog, brands,P_florida64_diff , storeweek,share)
dt, shares_florida64=est_ms_cm(d_florida64, th2, data_florida64, simpack)

data_MinMade64=(promosize, demog, brands, P_MinMade64_diff, storeweek,share)
dc, shares_MinMade64=est_ms_cm(d_MinMade64, th2, data_MinMade64, simpack)

data_HH64=(promosize, demog, brands, P_HH64_diff, storeweek,share)
dw, shares_HH64=est_ms_cm(d_HH64, th2, data_HH64, simpack)

data_Tropic64=(promosize, demog, brands, P_Tropic64_diff, storeweek,share)
da, shares_Tropic64=est_ms_cm(d_Tropic64, th2, data_Tropic64, simpack)

data_TropicSB64=(promosize, demog, brands, P_TropicSB64_diff, storeweek,share)
ds, shares_TropicSB64=est_ms_cm(d_TropicSB64, th2, data_TropicSB64, simpack)

data_MinMade96=(promosize, demog, brands, P_MinMade96_diff, storeweek,share)
dc, shares_MinMade96=est_ms_cm(d_MinMade96, th2, data_MinMade96, simpack)

data_HH96=(promosize, demog, brands, P_HH96_diff, storeweek,share)
dw, shares_HH96=est_ms_cm(d_HH96, th2, data_HH96, simpack)

data_Tropic96=(promosize, demog, brands, P_Tropic96_diff, storeweek,share)
da, shares_Tropic96=est_ms_cm(d_Tropic96, th2, data_Tropic96, simpack)


In [67]:
def filter_1(dif,dataset, filter1, value1, filter2, value2,col_groupby):
    data_i=dataset.loc[lambda di: dataset[filter1]==value1,:]
    data_m=data_i.loc[lambda df: data_i[filter2]==value2,:]
    data_f=data_m.groupby([col_groupby]).sum()
    data_f.iloc[:,11]=(data_f.iloc[:,11]-data_f.iloc[:,10])/dif
    data_f.iloc[:,12]=(data_f.iloc[:,12]-data_f.iloc[:,10])/dif
    data_f.iloc[:,13]=(data_f.iloc[:,13]-data_f.iloc[:,10])/dif
    data_f.iloc[:,14]=(data_f.iloc[:,14]-data_f.iloc[:,10])/dif
    data_f.iloc[:,15]=(data_f.iloc[:,15]-data_f.iloc[:,10])/dif
    data_f.iloc[:,16]=(data_f.iloc[:,16]-data_f.iloc[:,10])/dif
    data_f.iloc[:,17]=(data_f.iloc[:,17]-data_f.iloc[:,10])/dif
    data_f.iloc[:,18]=(data_f.iloc[:,18]-data_f.iloc[:,10])/dif
    return data_f

In [68]:
storeweek.shape#, promosize, brands, totcount, est_sh_test.shape

(29893, 1)

In [69]:
# generating the filtered data, each row in the 'share_x' columns show the effect in the 'brand' and 'size' market share 
# of a change in the price of '_x'

zone_week=np.array(datapack_GMM['zone'].apply(lambda x: x*1000)+datapack_GMM['week'], ndmin=2).T
week=np.array(datapack_GMM['week'], ndmin=2).T
totcount = np.array(datapack_GMM['totcount'], ndmin=2).T

q_diff = np.hstack((zone_week, week, storeweek, promosize, brands, totcount, est_sh_test, \
                    shares_florida64, shares_MinMade64, shares_MinMade96, \
                    shares_HH64, shares_HH96, shares_Tropic64, shares_Tropic96, shares_TropicSB64))

q_diff=pd.DataFrame(q_diff)

q_diff.columns = ['zone-week', 'week', 'store-week', 'promo', '64_OZ', 'florida','MinMade','HH','Tropic','TropicSB'\
                  , 'totcount', 'est_sh_eq', 'shares_pf64', 'shares_pm64'\
                  , 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64']

h=1e-5

sh_florida_64=filter_1(h, q_diff, 'florida', 1, '64_OZ', 1, 'store-week')
sh_florida_64['store-week']=sh_florida_64.index.values

sh_MinMade_64=filter_1(h, q_diff, 'MinMade', 1, '64_OZ', 1, 'store-week')
sh_MinMade_64['store-week']=sh_MinMade_64.index.values

sh_MinMade_96=filter_1(h, q_diff, 'MinMade', 1, '64_OZ', 0, 'store-week')
sh_MinMade_96['store-week']=sh_MinMade_96.index.values

sh_HH_64=filter_1(h, q_diff, 'HH', 1, '64_OZ', 1, 'store-week')
sh_HH_64['store-week']=sh_HH_64.index.values

sh_HH_96=filter_1(h, q_diff, 'HH', 1, '64_OZ', 0, 'store-week')
sh_HH_96['store-week']=sh_HH_96.index.values

sh_Tropic_64=filter_1(h, q_diff, 'Tropic', 1, '64_OZ', 1, 'store-week')
sh_Tropic_64['store-week']=sh_Tropic_64.index.values

sh_Tropic_96=filter_1(h, q_diff, 'Tropic', 1, '64_OZ', 0, 'store-week')
sh_Tropic_96['store-week']=sh_Tropic_96.index.values

sh_TropicSB_64=filter_1(h, q_diff, 'TropicSB', 1, '64_OZ', 1, 'store-week')
sh_TropicSB_64['store-week']=sh_TropicSB_64.index.values

In [70]:
sh_florida_64.iloc[:2,:]

,zone-week,week,promo,64_OZ,florida,MinMade,HH,Tropic,TropicSB,totcount,est_sh_eq,shares_pf64,shares_pm64,shares_pm96,shares_ph64,shares_ph96,shares_pt64,shares_pt96,shares_ptsb64,store-week
store-week,,,,,,,,,,,,,,,,,,,,
2173.000000,1173.000000,173.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,12672.000000,0.001140,0.001043,-0.000016,-0.000004,-0.000026,-0.000010,-0.000023,-0.000003,-0.000015,2173.000000
2174.000000,1174.000000,174.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,11477.000000,0.003003,0.002519,-0.000031,-0.000005,-0.000045,-0.000013,-0.000041,-0.000007,-0.000014,2174.000000


In [71]:
def weighted_prices(data, filter1, valuef1, filter2, valuef2, price1, price2, weight_col,groupby_col):
    prices_ini=data.loc[lambda pcs: data[filter1]==valuef1,:]
    prices_med=prices_ini.loc[lambda pcm: prices_ini[filter2]==valuef2,:]
    prices_med['weighted_price'] = prices_med[price1]*prices_med[weight_col]
    prices_med['weighted_wprice'] = prices_med[price2]*prices_med[weight_col]
    prices_fin = prices_med.groupby([groupby_col]).sum()
    prices_fin['weighted_price'] = prices_fin['weighted_price']/prices_fin[weight_col]
    prices_fin['weighted_wprice'] = prices_fin['weighted_wprice']/prices_fin[weight_col]
#    prices_fin[groupby_col] = prices_fin[groupby_col]/prices_fin[filter1]
    return prices_fin

In [72]:
#It could be possible that a product had not been sold in a store-week, 
#so for pricing it is necessary to exactly math the cuantities with the respectives products

#calculating weighted average store-week price and weighted average store-week wholesale price. This data also includes the
#Q total per brand, size, and store-week

move=np.array(datapack_GMM[['move']])

price_data=np.hstack((zone_week, storeweek, week, promosize, move, brands, price_est, wholeprice))
price_data=pd.DataFrame(price_data)

price_data.columns = ['zone-week', 'store-week', 'week', 'promo', '64_OZ', 'move', 'florida','MinMade','HH','Tropic',\
                      'TropicSB', 'price_est', 'wholeprice']

florida64_prices = weighted_prices(price_data, 'florida', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'store-week')
florida64_prices['store-week']=florida64_prices.index.values

MinMade64_prices = weighted_prices(price_data, 'MinMade', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'store-week')
MinMade64_prices['store-week']=MinMade64_prices.index.values

MinMade96_prices = weighted_prices(price_data, 'MinMade', 1, '64_OZ', 0, 'price_est', 'wholeprice', 'move', 'store-week')
MinMade96_prices['store-week']=MinMade96_prices.index.values

HH64_prices = weighted_prices(price_data, 'HH', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'store-week')
HH64_prices['store-week']=HH64_prices.index.values

HH96_prices = weighted_prices(price_data, 'HH', 1, '64_OZ', 0, 'price_est', 'wholeprice', 'move', 'store-week')
HH96_prices['store-week']=HH96_prices.index.values

Tropic64_prices = weighted_prices(price_data, 'Tropic', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'store-week')
Tropic64_prices['store-week']=Tropic64_prices.index.values

Tropic96_prices = weighted_prices(price_data, 'Tropic', 1, '64_OZ', 0, 'price_est', 'wholeprice', 'move', 'store-week')
Tropic96_prices['store-week']=Tropic96_prices.index.values

TropicSB64_prices = weighted_prices(price_data, 'TropicSB', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'store-week')
TropicSB64_prices['store-week']=TropicSB64_prices.index.values

C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [73]:
market2=datapack_GMM['store-week'].drop_duplicates()

market2.shape

(3904,)

In [74]:
def mapping(data1, data2, common, col1, col2, col3, col4, col5, col6, col7, col8):
    m1=data1[common].map(data2.set_index(common)[col1])
    m2=data1[common].map(data2.set_index(common)[col2])
    m3=data1[common].map(data2.set_index(common)[col3])
    m4=data1[common].map(data2.set_index(common)[col4])
    m5=data1[common].map(data2.set_index(common)[col5])
    m6=data1[common].map(data2.set_index(common)[col6])
    m7=data1[common].map(data2.set_index(common)[col7])
    m8=data1[common].map(data2.set_index(common)[col8])
    mapping=pd.concat([m1, m2, m3, m4, m5, m6, m7, m8], axis=1)
    mapping.columns=['shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64']
    return mapping

In [75]:
market2=datapack_GMM['store-week'].drop_duplicates()

wholesale_prices_s=pd.DataFrame(datapack_GMM['store-week'])
wholesale_prices_s=wholesale_prices_s.drop_duplicates()
wholesale_prices_s['m64']=wholesale_prices_s['store-week'].map(MinMade64_prices.set_index('store-week')['weighted_wprice'])
list(wholesale_prices_s)

['store-week', 'm64']

In [76]:
#calculating equilibrium prices on store configuration

market2=datapack_GMM['store-week'].drop_duplicates()

wholesale_prices_s=pd.DataFrame(datapack_GMM['store-week'].drop_duplicates())
wholesale_prices_s['f64']=wholesale_prices_s['store-week'].map(florida64_prices.set_index('store-week')['weighted_wprice'])
wholesale_prices_s['m64']=wholesale_prices_s['store-week'].map(MinMade64_prices.set_index('store-week')['weighted_wprice'])
wholesale_prices_s['m96']=wholesale_prices_s['store-week'].map(MinMade96_prices.set_index('store-week')['weighted_wprice'])
wholesale_prices_s['h64']=wholesale_prices_s['store-week'].map(HH64_prices.set_index('store-week')['weighted_wprice'])
wholesale_prices_s['h96']=wholesale_prices_s['store-week'].map(HH96_prices.set_index('store-week')['weighted_wprice'])
wholesale_prices_s['t64']=wholesale_prices_s['store-week'].map(Tropic64_prices.set_index('store-week')['weighted_wprice'])
wholesale_prices_s['f96']=wholesale_prices_s['store-week'].map(Tropic96_prices.set_index('store-week')['weighted_wprice'])
wholesale_prices_s['tsb64']=wholesale_prices_s['store-week'].map(TropicSB64_prices.set_index('store-week')['weighted_wprice'])
wholesale_prices_s=wholesale_prices_s.drop(columns=['store-week'], axis=1)

elasticities_s=pd.DataFrame(datapack_GMM['store-week'].drop_duplicates())
es_f64=mapping(elasticities_s,sh_florida_64,'store-week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')
es_m64=mapping(elasticities_s,sh_MinMade_64,'store-week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')
es_m96=mapping(elasticities_s,sh_MinMade_96,'store-week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')
es_h64=mapping(elasticities_s,sh_HH_64,'store-week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')
es_h96=mapping(elasticities_s,sh_HH_96,'store-week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')
es_t64=mapping(elasticities_s,sh_Tropic_64,'store-week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')
es_t96=mapping(elasticities_s,sh_Tropic_96,'store-week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')
es_tsb64=mapping(elasticities_s,sh_TropicSB_64,'store-week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')

elasticities_s=pd.concat([elasticities_s, es_f64,es_m64, es_m96, es_h64,es_h96, es_t64,es_t96, es_tsb64], axis=1)
elasticities_s=elasticities_s.drop(columns=['store-week'], axis=1)

Quants_s=pd.DataFrame(datapack_GMM['store-week'].drop_duplicates())
Quants_s['f64']=Quants_s['store-week'].map(florida64_prices.set_index('store-week')['move'])
Quants_s['m64']=Quants_s['store-week'].map(MinMade64_prices.set_index('store-week')['move'])
Quants_s['m96']=Quants_s['store-week'].map(MinMade96_prices.set_index('store-week')['move'])
Quants_s['h64']=Quants_s['store-week'].map(HH64_prices.set_index('store-week')['move'])
Quants_s['h96']=Quants_s['store-week'].map(HH96_prices.set_index('store-week')['move'])
Quants_s['t64']=Quants_s['store-week'].map(Tropic64_prices.set_index('store-week')['move'])
Quants_s['f96']=Quants_s['store-week'].map(Tropic96_prices.set_index('store-week')['move'])
Quants_s['tsb64']=Quants_s['store-week'].map(TropicSB64_prices.set_index('store-week')['move'])
Quants_s=Quants_s.drop(columns=['store-week'], axis=1)

opt_price_ST = pd.concat([market2, wholesale_prices_s, elasticities_s, Quants_s], axis=1, sort=False)

opt_pricesST_matrix=np.zeros((market2.size,8))

for i in range(market2.shape[0]):
    omega=np.vstack([[opt_price_ST.iloc[i,9:17]],[opt_price_ST.iloc[i,17:25]],[opt_price_ST.iloc[i,25:33]]\
                     ,[opt_price_ST.iloc[i,33:41]],[opt_price_ST.iloc[i,41:49]],[opt_price_ST.iloc[i,49:57]]\
                     ,[opt_price_ST.iloc[i,57:65]],[opt_price_ST.iloc[i,65:73]]])
    omega_inv=lin.inv(omega)
    whpr=np.reshape(np.array(opt_price_ST.iloc[i,1:9]),(8,1)) - omega_inv@np.reshape(np.array(opt_price_ST.iloc[i,73:81]),(8,1))
    opt_pricesST_matrix[i,:]=whpr.T
    i+=1

market2=np.array(market2,ndmin=2).T

opt_pricesST_matrix=np.hstack((market2, opt_pricesST_matrix))

opt_pricesST=pd.DataFrame(opt_pricesST_matrix)
opt_pricesST.columns=['store-week','f64_s', 'm64_s', 'm96_s', 'h64_s', 'h96_s', 't64_s', 't96_s', 'tsb64_s']


### Zone configuration

What did I do?: i) to calculate dq/dp for each zone-week, using the weighted average per store shares (wieghted by totalcount), ii) to calculate weighted average prices and wholesale prices for zone-week, and, iii) using dp/dq, quantities and wholesale prices, calculate the equilibrium price for this configuration.

In [78]:
def filter_2(dif,dataset, filter1, value1, filter2, value2,col_groupby1, col_groupby2, weight):
    data_i=dataset.loc[lambda di: dataset[filter1]==value1,:]
    data_m=data_i.loc[lambda df: data_i[filter2]==value2,:]
    data_m=data_m.groupby([col_groupby1]).sum()
    data_m.iloc[:,11]=data_m.iloc[:,11]*data_m[weight]
    data_m.iloc[:,12]=data_m.iloc[:,12]*data_m[weight]
    data_m.iloc[:,13]=data_m.iloc[:,13]*data_m[weight]
    data_m.iloc[:,14]=data_m.iloc[:,14]*data_m[weight]
    data_m.iloc[:,15]=data_m.iloc[:,15]*data_m[weight]
    data_m.iloc[:,16]=data_m.iloc[:,16]*data_m[weight]
    data_m.iloc[:,17]=data_m.iloc[:,17]*data_m[weight]
    data_m.iloc[:,18]=data_m.iloc[:,18]*data_m[weight]
    data_f=data_m.groupby([col_groupby2]).sum()
    data_f.iloc[:,10]=data_f.iloc[:,10]*data_f[weight]
    data_f.iloc[:,11]=data_f.iloc[:,11]*data_f[weight]
    data_f.iloc[:,12]=data_f.iloc[:,12]*data_f[weight]
    data_f.iloc[:,13]=data_f.iloc[:,13]*data_f[weight]
    data_f.iloc[:,14]=data_f.iloc[:,14]*data_f[weight]
    data_f.iloc[:,15]=data_f.iloc[:,15]*data_f[weight]
    data_f.iloc[:,16]=data_f.iloc[:,16]*data_f[weight]
    data_f.iloc[:,17]=data_f.iloc[:,17]*data_f[weight]
#    data_f[col_groupby1] = data_f[col_groupby1]/data_f[filter1]
#    data_f[col_groupby2] = data_f[col_groupby2]/data_f[filter1]
    data_f.iloc[:,10]=(data_f.iloc[:,10]-data_f.iloc[:,9])/dif
    data_f.iloc[:,11]=(data_f.iloc[:,11]-data_f.iloc[:,9])/dif
    data_f.iloc[:,12]=(data_f.iloc[:,12]-data_f.iloc[:,9])/dif
    data_f.iloc[:,13]=(data_f.iloc[:,13]-data_f.iloc[:,9])/dif
    data_f.iloc[:,14]=(data_f.iloc[:,14]-data_f.iloc[:,9])/dif
    data_f.iloc[:,15]=(data_f.iloc[:,15]-data_f.iloc[:,9])/dif
    data_f.iloc[:,16]=(data_f.iloc[:,16]-data_f.iloc[:,9])/dif
    data_f.iloc[:,17]=(data_f.iloc[:,17]-data_f.iloc[:,9])/dif
    return data_f

In [79]:
sh_f64_zone=filter_2(h, q_diff, 'florida', 1, '64_OZ', 1, 'store-week', 'zone-week', 'totcount')
sh_f64_zone['zone-week']=sh_f64_zone.index.values

sh_m64_zone=filter_2(h, q_diff, 'MinMade', 1, '64_OZ', 1, 'store-week', 'zone-week', 'totcount')
sh_m64_zone['zone-week']=sh_m64_zone.index.values

sh_m96_zone=filter_2(h, q_diff, 'MinMade', 1, '64_OZ', 0, 'store-week', 'zone-week', 'totcount')
sh_m96_zone['zone-week']=sh_m96_zone.index.values

sh_h64_zone=filter_2(h, q_diff, 'HH', 1, '64_OZ', 1, 'store-week', 'zone-week', 'totcount')
sh_h64_zone['zone-week']=sh_h64_zone.index.values

sh_h96_zone=filter_2(h, q_diff, 'HH', 1, '64_OZ', 0, 'store-week', 'zone-week', 'totcount')
sh_h96_zone['zone-week']=sh_h96_zone.index.values

sh_t64_zone=filter_2(h, q_diff, 'Tropic', 1, '64_OZ', 1, 'store-week', 'zone-week', 'totcount')
sh_t64_zone['zone-week']=sh_t64_zone.index.values

sh_t96_zone=filter_2(h, q_diff, 'Tropic', 1, '64_OZ', 0, 'store-week', 'zone-week', 'totcount')
sh_t96_zone['zone-week']=sh_t96_zone.index.values

sh_tsb64_zone=filter_2(h, q_diff, 'TropicSB', 1, '64_OZ', 1, 'store-week', 'zone-week', 'totcount')
sh_tsb64_zone['zone-week']=sh_tsb64_zone.index.values

In [80]:
f64_prices_zone = weighted_prices(price_data, 'florida', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'zone-week')
f64_prices_zone['zone-week']=f64_prices_zone.index.values

m64_prices_zone = weighted_prices(price_data, 'MinMade', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'zone-week')
m64_prices_zone['zone-week']=m64_prices_zone.index.values

m96_prices_zone = weighted_prices(price_data, 'MinMade', 1, '64_OZ', 0, 'price_est', 'wholeprice', 'move', 'zone-week')
m96_prices_zone['zone-week']=m96_prices_zone.index.values

h64_prices_zone = weighted_prices(price_data, 'HH', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'zone-week')
h64_prices_zone['zone-week']=h64_prices_zone.index.values

h96_prices_zone = weighted_prices(price_data, 'HH', 1, '64_OZ', 0, 'price_est', 'wholeprice', 'move', 'zone-week')
h96_prices_zone['zone-week']=h96_prices_zone.index.values

t64_prices_zone = weighted_prices(price_data, 'Tropic', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'zone-week')
t64_prices_zone['zone-week']=t64_prices_zone.index.values

t96_prices_zone = weighted_prices(price_data, 'Tropic', 1, '64_OZ', 0, 'price_est', 'wholeprice', 'move', 'zone-week')
t96_prices_zone['zone-week']=t96_prices_zone.index.values

tsb64_prices_zone = weighted_prices(price_data, 'TropicSB', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'zone-week')
tsb64_prices_zone['zone-week']=tsb64_prices_zone.index.values


C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [81]:

#calculating equilibrium prices on zone configuration

zone_week=pd.DataFrame(zone_week)
market3=zone_week.drop_duplicates()


wholesale_prices_z=zone_week.drop_duplicates()
wholesale_prices_z.columns=['zone-week']
wholesale_prices_z['f64']=wholesale_prices_z['zone-week'].map(f64_prices_zone.set_index('zone-week')['weighted_wprice'])
wholesale_prices_z['m64']=wholesale_prices_z['zone-week'].map(m64_prices_zone.set_index('zone-week')['weighted_wprice'])
wholesale_prices_z['m96']=wholesale_prices_z['zone-week'].map(m96_prices_zone.set_index('zone-week')['weighted_wprice'])
wholesale_prices_z['h64']=wholesale_prices_z['zone-week'].map(h64_prices_zone.set_index('zone-week')['weighted_wprice'])
wholesale_prices_z['h96']=wholesale_prices_z['zone-week'].map(h96_prices_zone.set_index('zone-week')['weighted_wprice'])
wholesale_prices_z['t64']=wholesale_prices_z['zone-week'].map(t64_prices_zone.set_index('zone-week')['weighted_wprice'])
wholesale_prices_z['f96']=wholesale_prices_z['zone-week'].map(t96_prices_zone.set_index('zone-week')['weighted_wprice'])
wholesale_prices_z['tsb64']=wholesale_prices_z['zone-week'].map(tsb64_prices_zone.set_index('zone-week')['weighted_wprice'])
wholesale_prices_z=wholesale_prices_z.drop(columns=['zone-week'], axis=1)


elasticities_z=zone_week.drop_duplicates()
elasticities_z.columns=['zone-week']
ez_f64=mapping(elasticities_z,sh_f64_zone,'zone-week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')
ez_m64=mapping(elasticities_z,sh_m64_zone,'zone-week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')
ez_m96=mapping(elasticities_z,sh_m96_zone,'zone-week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')
ez_h64=mapping(elasticities_z,sh_h64_zone,'zone-week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')
ez_h96=mapping(elasticities_z,sh_h96_zone,'zone-week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')
ez_t64=mapping(elasticities_z,sh_t64_zone,'zone-week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')
ez_t96=mapping(elasticities_z,sh_t96_zone,'zone-week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')
ez_tsb64=mapping(elasticities_z,sh_tsb64_zone,'zone-week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')

elasticities_z=pd.concat([elasticities_z, ez_f64,ez_m64, ez_m96, ez_h64,ez_h96, ez_t64,ez_t96, ez_tsb64], axis=1)
elasticities_z=elasticities_z.drop(columns=['zone-week'], axis=1)

Quants_z=zone_week.drop_duplicates()
Quants_z.columns=['zone-week']
Quants_z['f64']=Quants_z['zone-week'].map(f64_prices_zone.set_index('zone-week')['move'])
Quants_z['m64']=Quants_z['zone-week'].map(m64_prices_zone.set_index('zone-week')['move'])
Quants_z['m96']=Quants_z['zone-week'].map(m96_prices_zone.set_index('zone-week')['move'])
Quants_z['h64']=Quants_z['zone-week'].map(h64_prices_zone.set_index('zone-week')['move'])
Quants_z['h96']=Quants_z['zone-week'].map(h96_prices_zone.set_index('zone-week')['move'])
Quants_z['t64']=Quants_z['zone-week'].map(t64_prices_zone.set_index('zone-week')['move'])
Quants_z['f96']=Quants_z['zone-week'].map(t96_prices_zone.set_index('zone-week')['move'])
Quants_z['tsb64']=Quants_z['zone-week'].map(tsb64_prices_zone.set_index('zone-week')['move'])
Quants_z=Quants_z.drop(columns=['zone-week'], axis=1)

opt_price_ZN = np.hstack((market3, wholesale_prices_z, elasticities_z, Quants_z))

opt_price_ZN = pd.DataFrame(opt_price_ZN)

opt_pricesZN_matrix=np.zeros((market3.size,8))
i=0
for i in range(market3.shape[0]):
    omegaZN=np.vstack([[opt_price_ZN.iloc[i,9:17]],[opt_price_ZN.iloc[i,17:25]],[opt_price_ZN.iloc[i,25:33]]\
                     ,[opt_price_ZN.iloc[i,33:41]],[opt_price_ZN.iloc[i,41:49]],[opt_price_ZN.iloc[i,49:57]]\
                     ,[opt_price_ZN.iloc[i,57:65]],[opt_price_ZN.iloc[i,65:73]]])
    omegaZN_inv=lin.inv(omegaZN)
    whprZN=np.reshape(np.array(opt_price_ZN.iloc[i,1:9]),(8,1)) - \
    omegaZN_inv@np.reshape(np.array(opt_price_ZN.iloc[i,73:81]),(8,1))
    opt_pricesZN_matrix[i,:]=whprZN.T
    i+=1

market3=np.array(market3,ndmin=2)

opt_pricesZN_matrix=np.hstack((market3, opt_pricesZN_matrix))

opt_pricesZN=pd.DataFrame(opt_pricesZN_matrix)
opt_pricesZN.columns=['zone-week','f64_z', 'm64_z', 'm96_z', 'h64_z', 'h96_z', 't64_z', 't96_z', 'tsb64_z']


C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

### Chain configuration

What did I do?: i) to calculate dq/dp for each week, using the weighted average per store shares (wieghted by totalcount), ii) to calculate weighted average prices and wholesale prices for  each week, and, iii) using dp/dq, quantities and wholesale prices, calculate the equilibrium price for this configuration.

In [82]:
sh_f64_chain=filter_2(h, q_diff, 'florida', 1, '64_OZ', 1, 'store-week', 'week', 'totcount')
sh_f64_chain['week']=sh_f64_chain.index.values

sh_m64_chain=filter_2(h, q_diff, 'MinMade', 1, '64_OZ', 1, 'store-week', 'week', 'totcount')
sh_m64_chain['week']=sh_m64_chain.index.values

sh_m96_chain=filter_2(h, q_diff, 'MinMade', 1, '64_OZ', 0, 'store-week', 'week', 'totcount')
sh_m96_chain['week']=sh_m96_chain.index.values

sh_h64_chain=filter_2(h, q_diff, 'HH', 1, '64_OZ', 1, 'store-week', 'week', 'totcount')
sh_h64_chain['week']=sh_h64_chain.index.values

sh_h96_chain=filter_2(h, q_diff, 'HH', 1, '64_OZ', 0, 'store-week', 'week', 'totcount')
sh_h96_chain['week']=sh_h96_chain.index.values

sh_t64_chain=filter_2(h, q_diff, 'Tropic', 1, '64_OZ', 1, 'store-week', 'week', 'totcount')
sh_t64_chain['week']=sh_t64_chain.index.values

sh_t96_chain=filter_2(h, q_diff, 'Tropic', 1, '64_OZ', 0, 'store-week', 'week', 'totcount')
sh_t96_chain['week']=sh_t96_chain.index.values

sh_tsb64_chain=filter_2(h, q_diff, 'TropicSB', 1, '64_OZ', 1, 'store-week', 'week', 'totcount')
sh_tsb64_chain['week']=sh_tsb64_chain.index.values

In [83]:
f64_prices_chain = weighted_prices(price_data, 'florida', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'week')
f64_prices_chain['week']=f64_prices_chain.index.values

m64_prices_chain = weighted_prices(price_data, 'MinMade', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'week')
m64_prices_chain['week']=m64_prices_chain.index.values

m96_prices_chain = weighted_prices(price_data, 'MinMade', 1, '64_OZ', 0, 'price_est', 'wholeprice', 'move', 'week')
m96_prices_chain['week']=m96_prices_chain.index.values

h64_prices_chain = weighted_prices(price_data, 'HH', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'week')
h64_prices_chain['week']=h64_prices_chain.index.values

h96_prices_chain = weighted_prices(price_data, 'HH', 1, '64_OZ', 0, 'price_est', 'wholeprice', 'move', 'week')
h96_prices_chain['week']=h96_prices_chain.index.values

t64_prices_chain = weighted_prices(price_data, 'Tropic', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'week')
t64_prices_chain['week']=t64_prices_chain.index.values

t96_prices_chain = weighted_prices(price_data, 'Tropic', 1, '64_OZ', 0, 'price_est', 'wholeprice', 'move', 'week')
t96_prices_chain['week']=t96_prices_chain.index.values

tsb64_prices_chain = weighted_prices(price_data, 'TropicSB', 1, '64_OZ', 1, 'price_est', 'wholeprice', 'move', 'week')
tsb64_prices_chain['week']=tsb64_prices_chain.index.values

C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [84]:
#calculating equilibrium prices on chain configuration

week=pd.DataFrame(week)
market4=week.drop_duplicates()

wholesale_prices_c=week.drop_duplicates()
wholesale_prices_c.columns=['week']
wholesale_prices_c['f64']=wholesale_prices_c['week'].map(f64_prices_chain.set_index('week')['weighted_wprice'])
wholesale_prices_c['m64']=wholesale_prices_c['week'].map(m64_prices_chain.set_index('week')['weighted_wprice'])
wholesale_prices_c['m96']=wholesale_prices_c['week'].map(m96_prices_chain.set_index('week')['weighted_wprice'])
wholesale_prices_c['h64']=wholesale_prices_c['week'].map(h64_prices_chain.set_index('week')['weighted_wprice'])
wholesale_prices_c['h96']=wholesale_prices_c['week'].map(h96_prices_chain.set_index('week')['weighted_wprice'])
wholesale_prices_c['t64']=wholesale_prices_c['week'].map(t64_prices_chain.set_index('week')['weighted_wprice'])
wholesale_prices_c['f96']=wholesale_prices_c['week'].map(t96_prices_chain.set_index('week')['weighted_wprice'])
wholesale_prices_c['tsb64']=wholesale_prices_c['week'].map(tsb64_prices_chain.set_index('week')['weighted_wprice'])
wholesale_prices_c=wholesale_prices_c.drop(columns=['week'], axis=1)

elasticities_c=week.drop_duplicates()
elasticities_c.columns=['week']
ec_f64=mapping(elasticities_c,sh_f64_chain,'week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')
ec_m64=mapping(elasticities_c,sh_m64_chain,'week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')
ec_m96=mapping(elasticities_c,sh_m96_chain,'week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')
ec_h64=mapping(elasticities_c,sh_h64_chain,'week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')
ec_h96=mapping(elasticities_c,sh_h96_chain,'week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')
ec_t64=mapping(elasticities_c,sh_t64_chain,'week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')
ec_t96=mapping(elasticities_c,sh_t96_chain,'week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')
ec_tsb64=mapping(elasticities_c,sh_tsb64_chain,'week','shares_pf64', 'shares_pm64', 'shares_pm96', 'shares_ph64', 'shares_ph96', 'shares_pt64', 'shares_pt96', 'shares_ptsb64')

elasticities_c=pd.concat([elasticities_c, ec_f64,ec_m64, ec_m96, ec_h64,ec_h96, ec_t64,ec_t96, ec_tsb64], axis=1)
elasticities_c=elasticities_c.drop(columns=['week'], axis=1)

Quants_c=week.drop_duplicates()
Quants_c.columns=['week']
Quants_c['f64']=Quants_c['week'].map(f64_prices_chain.set_index('week')['move'])
Quants_c['m64']=Quants_c['week'].map(m64_prices_chain.set_index('week')['move'])
Quants_c['m96']=Quants_c['week'].map(m96_prices_chain.set_index('week')['move'])
Quants_c['h64']=Quants_c['week'].map(h64_prices_chain.set_index('week')['move'])
Quants_c['h96']=Quants_c['week'].map(h96_prices_chain.set_index('week')['move'])
Quants_c['t64']=Quants_c['week'].map(t64_prices_chain.set_index('week')['move'])
Quants_c['f96']=Quants_c['week'].map(t96_prices_chain.set_index('week')['move'])
Quants_c['tsb64']=Quants_c['week'].map(tsb64_prices_chain.set_index('week')['move'])
Quants_c=Quants_c.drop(columns=['week'], axis=1)


opt_price_CH = pd.concat([market4, wholesale_prices_c, elasticities_c, Quants_c], axis=1)

opt_pricesCH_matrix=np.zeros((market4.size,8))

for i in range(market4.shape[0]):
    omegaCH=np.vstack([[opt_price_CH.iloc[i,9:17]],[opt_price_CH.iloc[i,17:25]],[opt_price_CH.iloc[i,25:33]]\
                     ,[opt_price_CH.iloc[i,33:41]],[opt_price_CH.iloc[i,41:49]],[opt_price_CH.iloc[i,49:57]]\
                     ,[opt_price_CH.iloc[i,57:65]],[opt_price_CH.iloc[i,65:73]]])
    omegaCH_inv=lin.inv(omegaCH)
    whprCH=np.reshape(np.array(opt_price_CH.iloc[i,1:9]),(8,1)) - \
    omegaCH_inv@np.reshape(np.array(opt_price_CH.iloc[i,73:81]),(8,1))
    opt_pricesCH_matrix[i,:]=whprCH.T
    i+=1

opt_pricesCH_matrix=np.hstack((market4, opt_pricesCH_matrix))

opt_pricesCH=pd.DataFrame(opt_pricesCH_matrix)
opt_pricesCH.columns=['week','f64_c', 'm64_c', 'm96_c', 'h64_c', 'h96_c', 't64_c', 't96_c', 'tsb64_c']


C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\Daniela\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

# Customer Welfare

## Chain vs Store

What did I do?: i) ...

In [85]:
market3.shape

(765, 1)

In [86]:
# Building the store-week dataset.


datapack_cws = np.hstack((zone_week, datapack_GMM[['zone', 'week', 'store', 'store-week','totcount', 'move', 'promo',\
                                            '64 OZ', '96 OZ', 'florida','MinMade','HH','Tropic','TropicSB']]))

datapack_cws=pd.DataFrame(datapack_cws)

datapack_cws.columns = ['zone-week', 'zone', 'week', 'store', 'store-week','totcount', 'move', 'promo',\
                        '64 OZ', '96 OZ', 'florida','MinMade','HH','Tropic','TropicSB']

datapack_cws['income']=datapack_cws['store'].map(zones.set_index('store')['income'])
datapack_cws['AGE60']=datapack_cws['store'].map(zones.set_index('store')['AGE60'])
datapack_cws['ethnic']=datapack_cws['store'].map(zones.set_index('store')['ethnic'])
datapack_cws['hvalmean']=datapack_cws['store'].map(zones.set_index('store')['hvalmean'])
datapack_cws['shopindx']=datapack_cws['store'].map(zones.set_index('store')['shopindx'])
datapack_cws['cubdist']=datapack_cws['store'].map(zones.set_index('store')['cubdist'])
datapack_cws['omnidist']=datapack_cws['store'].map(zones.set_index('store')['omnidist'])
datapack_cws['sstrdist']=datapack_cws['store'].map(zones.set_index('store')['sstrdist'])
datapack_cws['dtdist']=datapack_cws['store'].map(zones.set_index('store')['dtdist'])

datapack_cws['pf64'] = datapack_cws['store-week'].map(opt_pricesST.set_index('store-week')['f64_s'])*datapack_cws['64 OZ']*datapack_cws['florida']

datapack_cws['pm64'] = datapack_cws['store-week'].map(opt_pricesST.set_index('store-week')['m64_s'])*datapack_cws['64 OZ']*datapack_cws['MinMade']

datapack_cws['pm96'] = datapack_cws['store-week'].map(opt_pricesST.set_index('store-week')['m96_s'])*datapack_cws['96 OZ']*datapack_cws['MinMade']

datapack_cws['ph64'] = datapack_cws['store-week'].map(opt_pricesST.set_index('store-week')['h64_s'])*datapack_cws['64 OZ']*datapack_cws['HH']

datapack_cws['ph96'] = datapack_cws['store-week'].map(opt_pricesST.set_index('store-week')['h96_s'])*datapack_cws['96 OZ']*datapack_cws['HH']

datapack_cws['pt64'] = datapack_cws['store-week'].map(opt_pricesST.set_index('store-week')['t64_s'])*datapack_cws['64 OZ']*datapack_cws['Tropic']

datapack_cws['pt96'] = datapack_cws['store-week'].map(opt_pricesST.set_index('store-week')['t96_s'])*datapack_cws['96 OZ']*datapack_cws['Tropic']

datapack_cws['ptsb64'] = datapack_cws['store-week'].map(opt_pricesST.set_index('store-week')['tsb64_s'])*datapack_cws['64 OZ']*datapack_cws['TropicSB']

In [87]:
datapack_cws.iloc[:5,20:]

,cubdist,omnidist,sstrdist,dtdist,pf64,pm64,pm96,ph64,ph96,pt64,pt96,ptsb64
0,3.784829,2.110122,2.110122,1.702432,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-93042.408591
1,3.784829,2.110122,2.110122,1.702432,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-37718.492917,-0.000000
2,3.784829,2.110122,2.110122,1.702432,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-15672.874545,-0.000000,-0.000000
3,3.784829,2.110122,2.110122,1.702432,-0.000000,-0.000000,-0.000000,-0.000000,-13306.889406,-0.000000,-0.000000,-0.000000
4,3.784829,2.110122,2.110122,1.702432,-0.000000,-0.000000,-0.000000,-14578.238117,-0.000000,-0.000000,-0.000000,-0.000000


In [88]:
# Building the zone-week dataset.


datapack_cwz = np.hstack((zone_week, datapack_GMM[['zone', 'week', 'store', 'store-week','totcount', 'move', 'promo',\
                                            '64 OZ', '96 OZ', 'florida','MinMade','HH','Tropic','TropicSB']]))

datapack_cwz=pd.DataFrame(datapack_cwz)

datapack_cwz.columns = ['zone-week', 'zone', 'week', 'store', 'store-week','totcount', 'move', 'promo',\
                        '64 OZ', '96 OZ', 'florida','MinMade','HH','Tropic','TropicSB']


datapack_cwz['income']=datapack_cwz['store'].map(zones.set_index('store')['income'])#*datapack_cwz['totcount']
datapack_cwz['AGE60']=datapack_cwz['store'].map(zones.set_index('store')['AGE60'])#*datapack_cwz['totcount']
datapack_cwz['ethnic']=datapack_cwz['store'].map(zones.set_index('store')['ethnic'])#*datapack_cwz['totcount']
datapack_cwz['hvalmean']=datapack_cwz['store'].map(zones.set_index('store')['hvalmean'])#*datapack_cwz['totcount']
datapack_cwz['shopindx']=datapack_cwz['store'].map(zones.set_index('store')['shopindx'])#*datapack_cwz['totcount']
datapack_cwz['cubdist']=datapack_cwz['store'].map(zones.set_index('store')['cubdist'])#*datapack_cwz['totcount']
datapack_cwz['omnidist']=datapack_cwz['store'].map(zones.set_index('store')['omnidist'])#*datapack_cwz['totcount']
datapack_cwz['sstrdist']=datapack_cwz['store'].map(zones.set_index('store')['sstrdist'])#*datapack_cwz['totcount']
datapack_cwz['dtdist']=datapack_cwz['store'].map(zones.set_index('store')['dtdist'])#*datapack_cwz['totcount']


index_cwz = datapack_cwz.index.values

datapack_cwz['pf64'] = f64_prices_zone.loc[index_cwz[:],'weighted_price']*datapack_cwz['64 OZ']*datapack_cwz['florida']

datapack_cwz['pm64'] = m64_prices_zone.loc[index_cwz[:],'weighted_price']*datapack_cwz['64 OZ']*datapack_cwz['MinMade']

datapack_cwz['pm96'] = m96_prices_zone.loc[index_cwz[:],'weighted_price']*datapack_cwz['96 OZ']*datapack_cwz['MinMade']

datapack_cwz['ph64'] = h64_prices_zone.loc[index_cwz[:],'weighted_price']*datapack_cwz['64 OZ']*datapack_cwz['HH']

datapack_cwz['ph96'] = h96_prices_zone.loc[index_cwz[:],'weighted_price']*datapack_cwz['96 OZ']*datapack_cwz['HH']

datapack_cwz['pt64'] = h64_prices_zone.loc[index_cwz[:],'weighted_price']*datapack_cwz['64 OZ']*datapack_cwz['Tropic']

datapack_cwz['pt96'] = h96_prices_zone.loc[index_cwz[:],'weighted_price']*datapack_cwz['96 OZ']*datapack_cwz['Tropic']

datapack_cwz['ptsb64'] = tsb64_prices_zone.loc[index_cwz[:],'weighted_price']*datapack_cwz['64 OZ']*datapack_cwz['TropicSB']

C:\Users\Daniela\Anaconda3\lib\site-packages\pandas\core\indexing.py:1027: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


In [89]:
datapack_cwz.shape

(29893, 32)

In [90]:
# Building the chain-week dataset.


datapack_cwc = np.hstack((zone_week, datapack_GMM[['zone', 'week', 'store', 'store-week','totcount', 'move', 'promo',\
                                            '64 OZ', '96 OZ', 'florida','MinMade','HH','Tropic','TropicSB']]))

datapack_cwc=pd.DataFrame(datapack_cwc)

datapack_cwc.columns = ['zone-week', 'zone', 'week', 'store', 'store-week','totcount', 'move', 'promo',\
                        '64 OZ', '96 OZ', 'florida','MinMade','HH','Tropic','TropicSB']

 
datapack_cwc['income']=datapack_cwc['store'].map(zones.set_index('store')['income'])#*datapack_cwc['totcount']
datapack_cwc['AGE60']=datapack_cwc['store'].map(zones.set_index('store')['AGE60'])#*datapack_cwc['totcount']
datapack_cwc['ethnic']=datapack_cwc['store'].map(zones.set_index('store')['ethnic'])#*datapack_cwc['totcount']
datapack_cwc['hvalmean']=datapack_cwc['store'].map(zones.set_index('store')['hvalmean'])#*datapack_cwc['totcount']
datapack_cwc['shopindx']=datapack_cwc['store'].map(zones.set_index('store')['shopindx'])#*datapack_cwc['totcount']
datapack_cwc['cubdist']=datapack_cwc['store'].map(zones.set_index('store')['cubdist'])#*datapack_cwc['totcount']
datapack_cwc['omnidist']=datapack_cwc['store'].map(zones.set_index('store')['omnidist'])#*datapack_cwc['totcount']
datapack_cwc['sstrdist']=datapack_cwc['store'].map(zones.set_index('store')['sstrdist'])#*datapack_cwc['totcount']
datapack_cwc['dtdist']=datapack_cwc['store'].map(zones.set_index('store')['dtdist'])#*datapack_cwc['totcount']


index_cwc = datapack_cwc.index.values

datapack_cwc['pf64'] = f64_prices_chain.loc[index_cwc[:],'weighted_price']*datapack_cwc['64 OZ']*datapack_cwc['florida']

datapack_cwc['pm64'] = m64_prices_chain.loc[index_cwc[:],'weighted_price']*datapack_cwc['64 OZ']*datapack_cwc['MinMade']

datapack_cwc['pm96'] = m96_prices_chain.loc[index_cwc[:],'weighted_price']*datapack_cwc['96 OZ']*datapack_cwc['MinMade']

datapack_cwc['ph64'] = h64_prices_chain.loc[index_cwc[:],'weighted_price']*datapack_cwc['64 OZ']*datapack_cwc['HH']

datapack_cwc['ph96'] = h96_prices_chain.loc[index_cwc[:],'weighted_price']*datapack_cwc['96 OZ']*datapack_cwc['HH']

datapack_cwc['pt64'] = t64_prices_chain.loc[index_cwc[:],'weighted_price']*datapack_cwc['64 OZ']*datapack_cwc['Tropic']

datapack_cwc['pt96'] = t96_prices_chain.loc[index_cwc[:],'weighted_price']*datapack_cwc['96 OZ']*datapack_cwc['Tropic']

datapack_cwc['ptsb64'] = tsb64_prices_chain.loc[index_cwc[:],'weighted_price']*datapack_cwc['64 OZ']*datapack_cwc['TropicSB']

## Calculating differences in consumer welfare
What did I do?: i) prepare the data to generate the deltas and the mu. ii) calculate the deltas and mu for each configuration. iii) group by store-week. iv) take logs. v) calculate the numerator. vi) divide by $\theta$. vii) take average across simulations

In [92]:
# Preparing the data

# draws:

#normal_beta_draws
#normal_theta_draws
#normal_w_draws

# store configuration:

promo_size_s = np.array(datapack_cws[['promo', '64 OZ']])
demog_data_s = np.array(datapack_cws[['income', 'AGE60', 'ethnic', 'hvalmean','shopindx','sstrdist']])
brand_dummies_s = np.array(datapack_cws[['florida','MinMade','HH','Tropic','TropicSB']])
storeweek_s = np.array(datapack_cws['store-week'])

price1_s = datapack_cws[['pf64', 'pm64', 'pm96', 'ph64', 'ph96', 'pt64', 'pt96', 'ptsb64']]
price2_s = price1_s.sum(axis=1)
price_s = np.array(price2_s, ndmin=2).T

# zone configuration:

promo_size_z = np.array(datapack_cwz[['promo', '64 OZ']])
demog_data_z = np.array(datapack_cwz[['income', 'AGE60', 'ethnic', 'hvalmean','shopindx','sstrdist']])
brand_dummies_z = np.array(datapack_cwz[['florida','MinMade','HH','Tropic','TropicSB']])
storeweek_z = np.array(datapack_cwz['store-week'])

price1_z = datapack_cwz[['pf64', 'pm64', 'pm96', 'ph64', 'ph96', 'pt64', 'pt96', 'ptsb64']]
price2_z = price1_z.sum(axis=1)
price_z = np.array(price2_z, ndmin=2).T

# chain configuration:

promo_size_c = np.array(datapack_cwc[['promo', '64 OZ']])
demog_data_c = np.array(datapack_cwc[['income', 'AGE60', 'ethnic', 'hvalmean','shopindx','sstrdist']])
brand_dummies_c = np.array(datapack_cwc[['florida','MinMade','HH','Tropic','TropicSB']])
storeweek_c = np.array(datapack_cwc['store-week'])

price1_c = datapack_cwc[['pf64', 'pm64', 'pm96', 'ph64', 'ph96', 'pt64', 'pt96', 'ptsb64']]
price2_c = price1_c.sum(axis=1)
price_c = np.array(price2_c, ndmin=2).T


In [93]:
# calculating deltas and mu for each configuration:

# draws:

#normal_beta_draws
#normal_theta_draws
#normal_w_draws

# store configuration:

xi_s=np.zeros((datapack_cws.shape[0],1))

delta_s = delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi_s, \
                     promo_size_s, demog_data_s, brand_dummies_s, price_s, storeweek_s)

mu_s = mu_fun(promo_size_s, price_s, brand_dummies_s, L_matrix_def, lambda_p_s_def,\
              lambda_price_def, normal_beta_draws, normal_theta_draws, normal_w_draws)

# zone configuration:

xi_z=np.zeros((datapack_cwz.shape[0],1))

delta_z = delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi_z, \
                     promo_size_z, demog_data_z, brand_dummies_z, price_z, storeweek_z)

mu_z = mu_fun(promo_size_z, price_z, brand_dummies_z, L_matrix_def, lambda_p_s_def,\
              lambda_price_def, normal_beta_draws, normal_theta_draws, normal_w_draws)

# chain configuration:

xi_c=np.zeros((datapack_cwc.shape[0],1))

delta_c = delta_fun(alpha_def, beta_def, sigma_def, gamma_def, theta_def, xi_c, \
                     promo_size_c, demog_data_c, brand_dummies_c, price_c, storeweek_c)

mu_c = mu_fun(promo_size_c, price_c, brand_dummies_c, L_matrix_def, lambda_p_s_def,\
              lambda_price_def, normal_beta_draws, normal_theta_draws, normal_w_draws)

In [1]:
# adding up, taking exponential, grouping by store_week and taking logs

# store configuration

delta_s2=np.array(delta_s[:,1], ndmin=2).T

exp_ut1_s = delta_s2 + mu_s
exp_ut2_s = np.exp(exp_ut1_s)
exp_ut2_s = pd.DataFrame(exp_ut2_s)
exp_ut2_s = 1 + exp_ut2_s
exp_ut2_s['store-week'] = datapack_cws['store-week']
exp_ut3_s = exp_ut2_s.groupby(['store-week']).sum()
exp_ut3_s = np.log(exp_ut3_s)

log_expected_ut_s = exp_ut3_s

# zone configuration

delta_z2=np.array(delta_z[:,1], ndmin=2).T

exp_ut1_z = delta_z2 + mu_z
exp_ut2_z = np.exp(exp_ut1_z)
exp_ut2_z = pd.DataFrame(exp_ut2_z)
exp_ut2_z = 1 + exp_ut2_z
exp_ut2_z['store-week'] = datapack_cwz['store-week']
exp_ut3_z = exp_ut2_z.groupby(['store-week']).sum()
exp_ut3_z = np.log(exp_ut3_z)

log_expected_ut_z = exp_ut3_z

# chain configuration

delta_c2=np.array(delta_c[:,1], ndmin=2).T

exp_ut1_c = delta_c2 + mu_c
exp_ut2_c = np.exp(exp_ut1_c)
exp_ut2_c = pd.DataFrame(exp_ut2_c)
exp_ut2_c = 1 + exp_ut2_c
exp_ut2_c['store-week'] = datapack_cwc['store-week2']
exp_ut3_c = exp_ut2_c.groupby(['store-week']).sum()
exp_ut3_c = np.log(exp_ut2_c)

log_expected_ut_c = exp_ut3_c

#theta vector (1 row x 30 columns)

gamma = np.reshape(gamma_def, (gamma_def.size,1))
normal_theta_draws = np.reshape(normal_theta_draws,(1, normal_theta_draws.size))

theta_vec = theta_def + demog_data_s@gamma + lambda_price_def*normal_theta_draws
#review dimensions of theta vec

NameError: name 'np' is not defined

In [98]:
# calculating differences in consumer welfare per store week, and integrating across simulations

#(share_mat2.iloc[:,1:].sum(axis=1)).apply(lambda x: x*(1/ns))

nsim=normal_theta_draws.shape[1]
Mt = np.array([datapack_GMM['totcount']], ndmin=2).T

i=0
Comp_Variation_sc = np.zeros((Mt.size, 1))#[0]*log_expected_ut_c.shape[0]

for i in range(log_expected_ut_c.shape[0]):
    Comp_Variation_sc[i] = ((log_expected_ut_s[i,:]-log_expected_ut_c[i,:])/theta_vec[i,:]).sum()*(1/nsim)
    i+=1

i=0
Comp_Variation_zc = np.zeros((Mt.size, 1))#[0]*log_expected_ut_c.shape[0]

for i in range(log_expected_ut_c.shape[0]):
    Comp_Variation_zc[i] = ((log_expected_ut_z[i,:]-log_expected_ut_c[i,:])/theta_vec[i,:]).sum()*(1/nsim)
    i+=1

#DROPPING NAN AND INF VALUES

CV_CS = Comp_Variation_sc*Mt
CV_CS = pd.DataFrame(CV_CS)
CV_CS = CV_CS.replace([np.inf, -np.inf], np.nan)
CV_CS = CV_CS.dropna()
CV_CS = np.array(CV_CS)
Comp_Var_CS = CV_CS.sum()

CV_CZ = Comp_Variation_zc*Mt
CV_CZ = pd.DataFrame(CV_CZ)
CV_CZ = CV_CZ.replace([np.inf, -np.inf], np.nan)
CV_CZ = CV_CZ.dropna()
CV_CZ = np.array(CV_CZ)
Comp_Var_CZ = CV_CZ.sum()

In [99]:
print("Compensated Variation from Chain to Store:",Comp_Var_CS)
print("Compensated Variation from Chain toZone:",Comp_Var_CZ)

Compensated Variation from Chain to Store: -7583103751.111582
Compensated Variation from Chain toZone: 39381.836989612144
